In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [63]:
df=pd.read_csv('DM2425_ABCDEats_DATASET.csv',delimiter=',')

#Drop Dublicates

In [64]:
df.duplicated().sum()

13

In [65]:
df=df.drop_duplicates()
# Count the number of remaining duplicate rows in the DataFrame after dropping duplicates
df.duplicated().sum()

0

# Set costumer_id as the index

In [66]:
df.set_index('customer_id', inplace=True)

## Fixing variables

**costumer_age**

Fill missing values with median

In [67]:
df['customer_age'].isna().sum()

727

In [68]:
df['customer_age'].fillna(df['customer_age'].median(),inplace=True)
df['customer_age'].isna().sum()

<ipython-input-68-3ad8a2b86644>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['customer_age'].fillna(df['customer_age'].median(),inplace=True)
<ipython-input-68-3ad8a2b86644>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_age'].fillna(df['customer_age'].median(),inplace=True)


0

**costumer_region**

In [69]:
df['customer_region'] = df['customer_region'].replace('-', 'Unknown')

<ipython-input-69-0bbbfade363b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_region'] = df['customer_region'].replace('-', 'Unknown')


**product_count**

In [70]:
df.loc['92ced2dcb9','product_count']=100

**first_order**

In [71]:
df['first_order'].fillna(df['first_order'].median(),inplace=True)

<ipython-input-71-542326bf2f64>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['first_order'].fillna(df['first_order'].median(),inplace=True)


**last_promo**

In [72]:
df['last_promo'] = df['last_promo'].replace('-', 'no_promotion_used')

**HR_0**

In [73]:
df.drop(columns=['HR_0'], inplace=True)

#Fixing inconsistencies

Eliminate rows where there are inconsistencies

In [74]:
df['sum_HR']=df['HR_1']+df['HR_2']+df['HR_3']+df['HR_4']+df['HR_5']+df['HR_6']+df['HR_7']+df['HR_8']+df['HR_9']+df['HR_10']+df['HR_11']+df['HR_12']+df['HR_13']+df['HR_14']+df['HR_15']+df['HR_16']+df['HR_17']+df['HR_18']+df['HR_19']+df['HR_20']+df['HR_21']+df['HR_22']+df['HR_23']
df['sum_DOW']=df['DOW_0']+df['DOW_1']+df['DOW_2']+df['DOW_3']+df['DOW_4']+df['DOW_5']+df['DOW_6']
df['inc_0']=(df['sum_HR']!=df['sum_DOW'])
df['inc_2']=(df['product_count']==0)
df = df.loc[~(df['inc_0'] | df['inc_2'])]
df.drop(columns=['sum_HR', 'sum_DOW', 'inc_0', 'inc_2'], inplace=True)

<ipython-input-74-460a7041edd9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['sum_HR', 'sum_DOW', 'inc_0', 'inc_2'], inplace=True)


# New variables

**Total_Spent**

In [75]:
cuisine = [col for col in df.columns if col.startswith('CUI_')]

In [76]:
df['Total_Spent']=df[cuisine].sum(axis=1)

<ipython-input-76-e0b8285253ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total_Spent']=df[cuisine].sum(axis=1)


Deal with extreme outliers

In [77]:
df = df.drop('92ced2dcb9', axis=0)
df.loc['249ba584d3','CUI_Asian']=500
df.loc['8d65421423','CUI_Japanese']=590
df['Total_Spent']=df[cuisine].sum(axis=1)

**Order_Count**

In [78]:
df['order_count']=df['DOW_0']+df['DOW_1']+df['DOW_2']+df['DOW_3']+df['DOW_4']+df['DOW_5']+df['DOW_6']

**Diversity_Cuisine**

In [79]:
df['Diversity_Cuisine'] = (df[cuisine] > 0).sum(axis=1)
df['Diversity_Cuisine'] = df['order_count']/df['Diversity_Cuisine']

**Costumer_time**

In [80]:
df['costumer_time']=df['last_order']-df['first_order']

**Intensity_of_Activity**

In [81]:
df['Intensity_of_Activity'] = df['costumer_time'] / df['order_count']

**Customer_Loyalty**

In [82]:
df['Customer_loyalty'] = df['vendor_count'] / df['order_count']

**Product_Intensity**

In [85]:
df['product_intensity'] = df['product_count'] / df['order_count']